In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly"
from datetime import datetime, date
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import nevergrad as ng
import plotly.express as px

from estival.model import BayesianCompartmentalModel
from estival.optimization.nevergrad import optimize_model
from estival.priors import UniformPrior
from estival.targets import NegativeBinomialTarget, CustomTarget
from estival.calibration import pymc as epm
from tbdynamics import model

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
time_start = 1800
time_end = 2020
time_step = 1

In [3]:
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

In [18]:
params = {
    "contact_rate": 0.009,
    "start_population_size": 200000,
    "progression_multiplier": 2,
    "seed": 1,
    "rr_infection_latent": 0.5,
    "rr_infection_recovered": 0.5,
    "infect_death_rate_unstratified": 0.21
}

In [5]:
tb_model, build_text = model.build_base_model(compartments, infectious_compartments, time_start, time_end, time_step)

In [6]:
build_text

"The base model consists of 6 states, representing the following states: susceptible, early_latent, late_latent, infectious, on_treatment, recovered. Only the ['infectious', 'on_treatment'] compartment contributes to the force of infection. The model is run from 1800 to 2020. "

In [7]:
model.set_starting_conditions(tb_model, params["start_population_size"], params["seed"])



'The simulation starts with 0.2 million fully susceptible persons, with infectious persons introduced later through strain seeding as described below. '

In [8]:
model.add_infection(tb_model)

('The infection process moves people from the susceptible compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The infection_from_latent process moves people from the late_latent compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The infection_from_recovered process moves people from the late_latent compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ')

In [9]:
model.add_latency(tb_model)

('The stabilisation process moves people from the susceptible compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The early_activation process moves people from the susceptible compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The late_activation process moves people from the late_latent compartment to the infectious compartment, under the frequency-dependent transmission assumption. ')

In [10]:
model.add_treatment(tb_model)

('The early_activation process moves people from the on_treatment compartment to the recovered compartment, under the frequency-dependent transmission assumption. ',
 'The death_flow process moves people from the on_treatment compartment to the death, under the frequency-dependent transmission assumption. ',
 'The early_activation process moves people from the on_treatment compartment to the infectious compartment, under the frequency-dependent transmission assumption. ')

In [11]:
model.add_entry_flow(tb_model)

In [12]:
model.add_natural_death_flow(tb_model)

In [19]:
params

{'contact_rate': 0.009,
 'start_population_size': 200000,
 'progression_multiplier': 2,
 'seed': 1,
 'rr_infection_latent': 0.5,
 'rr_infection_recovered': 0.5,
 'infect_death_rate_unstratified': 0.21}

In [20]:
tb_model.run(parameters=params)

In [22]:
tb_model.get_outputs_df()

,susceptible,early_latent,late_latent,infectious,on_treatment,recovered
1800.0,1.999990e+05,0.000000e+00,0.0,1.000000e+00,0.0,0.0
1801.0,9.959095e+03,6.361933e+04,0.0,3.678793e-01,0.0,0.0
1802.0,4.966724e+02,2.657241e+04,0.0,1.353353e-01,0.0,0.0
1803.0,2.492060e+01,9.933639e+03,0.0,4.978711e-02,0.0,0.0
1804.0,1.304975e+00,3.662393e+03,0.0,1.831567e-02,0.0,0.0
...,...,...,...,...,...,...
2016.0,7.254125e-94,4.041821e-89,0.0,2.000899e-94,0.0,0.0
2017.0,2.661830e-94,1.488769e-89,0.0,7.369977e-95,0.0,0.0
2018.0,9.799295e-95,5.483751e-90,0.0,2.714608e-95,0.0,0.0
2019.0,3.609246e-95,2.019892e-90,0.0,9.998807e-96,0.0,0.0
